In [1]:
import requests
import pandas as pd
import time
import numpy as np
import re
import sys
import random

In [2]:
jeju_food = pd.read_csv('../data/공공데이터/제주특별자치도음식점목록(통합).csv')

In [3]:
jeju_food.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42602 entries, 0 to 42601
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     42602 non-null  object 
 1   업종구분대분류  42602 non-null  object 
 2   업종구분소분류  42599 non-null  object 
 3   인허가일자    42584 non-null  float64
 4   인허가취소일자  18 non-null     object 
 5   영업상태명    42602 non-null  object 
 6   상세영업상태명  42594 non-null  object 
 7   폐업일자     23435 non-null  float64
 8   휴업시작일자   0 non-null      float64
 9   휴업종료일자   0 non-null      float64
 10  재개업일자    18 non-null     float64
 11  소재지면적    41998 non-null  object 
 12  소재지전체주소  42420 non-null  object 
 13  도로명전체주소  27767 non-null  object 
 14  도로명우편번호  27433 non-null  object 
 15  데이터갱신일자  42584 non-null  object 
dtypes: float64(5), object(11)
memory usage: 5.2+ MB


In [4]:
jeju_food_1=jeju_food[jeju_food['영업상태명'] != '폐업']

In [12]:
jeju_food_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19157 entries, 0 to 41281
Data columns (total 16 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   사업장명     19157 non-null  object 
 1   업종구분대분류  19157 non-null  object 
 2   업종구분소분류  19155 non-null  object 
 3   인허가일자    19149 non-null  float64
 4   인허가취소일자  8 non-null      object 
 5   영업상태명    19157 non-null  object 
 6   상세영업상태명  19149 non-null  object 
 7   폐업일자     0 non-null      float64
 8   휴업시작일자   0 non-null      float64
 9   휴업종료일자   0 non-null      float64
 10  재개업일자    8 non-null      float64
 11  소재지면적    19148 non-null  object 
 12  소재지전체주소  19062 non-null  object 
 13  도로명전체주소  18925 non-null  object 
 14  도로명우편번호  18818 non-null  object 
 15  데이터갱신일자  19149 non-null  object 
dtypes: float64(5), object(11)
memory usage: 2.5+ MB


In [8]:
# 1차 검색 제주 000 
url = 'https://map.naver.com/v5/api/search'
params = {
    'calller': 'pcweb',
    'query': '',
    'type': 'all',
    'searchCoord': '126.56379330000001;33.24652849999992',
    'page': 1,
    'displayCount': 20,
    'isPlaceRecommendationReplace': True,
    'lang': 'ko'
}
tmp_dict = dict()
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'cookie': 'NNB=SFVP2LHVPD7WC; NDARK=N; _gcl_au=1.1.876992382.1652928740; _fbp=fb.1.1652928739593.996991756; _ga=GA1.2.1723077753.1655082737; MM_NEW=1; NFS=2; _tt_enable_cookie=1; _ttp=d30a8fad-c361-4bb2-a6d7-884e4f9ca1c6; NV_WETR_LOCATION_RGN_M="MDkxNDAxMDQ="; NV_WETR_LAST_ACCESS_RGN_M="MDkxNDAxMDQ="; nx_ssl=2; csrf_token=b22420dac145f9757a431121f5b4c25948818847853c526728a4d2219155c97e9b6490d088524f5e2d1f9fdd189f23d95bdd86f6e9ca8c8b07bf30c857e040c6; page_uid=d78aa3bb-613a-4eab-8035-b9c4712fa1de',
    'expires': 'Sat, 01 Jan 2000 00:00:00 GMT',
    'pragma': 'no-cache',
    'referer': 'https://map.naver.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}
params2 ={
    'lang' :'ko'
}
#n차 검색 제주도 주소 검색 후 해당 위치의 가게 정보를 조회 
url3 = 'https://map.naver.com/v5/api/addresses/place/'
params3= {
    'address':'',
    'lang':'ko',
    'page':1
}
headers2 = {
    'authority': 'map.naver.com',
    'accept': 'application/json, text/plain, */*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4',
    'cache-control': 'no-cache',
    'content-type': 'application/json',
    'cookie': 'NNB=SFVP2LHVPD7WC; NDARK=N; _gcl_au=1.1.876992382.1652928740; _fbp=fb.1.1652928739593.996991756; _ga=GA1.2.1723077753.1655082737; MM_NEW=1; NFS=2; _tt_enable_cookie=1; _ttp=d30a8fad-c361-4bb2-a6d7-884e4f9ca1c6; NV_WETR_LOCATION_RGN_M="MDkxNDAxMDQ="; NV_WETR_LAST_ACCESS_RGN_M="MDkxNDAxMDQ="; nx_ssl=2; page_uid=hXuWqwp0YiRssBM4BUGssssssMo-163853; BMR=s=1658907970093&r=https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=heygun&logNo=221520454294&r2=https://www.google.com/; page_uid=3c7d105d-de2b-4c43-8a74-1c2abebf354d',
    'expires': 'Sat, 01 Jan 2000 00:00:00 GMT',
    'pragma ': 'no-cache',
    'referer': 'https://map.naver.com/',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36'
}